# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [98]:
import keras
from keras.datasets import cifar10

In [99]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [100]:
import numpy as np
np.unique(y_train),np.unique(y_test)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8),
 array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))

In [101]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)


x_train shape: (50000, 32, 32, 3)
x_test shape: (10000, 32, 32, 3)
y_train shape: (50000, 1)
y_test shape: (10000, 1)


In [102]:
y_train=y_train.reshape(y_train.shape[0])
y_test=y_test.reshape(y_test.shape[0])

In [103]:
x_train=x_train.reshape(x_train.shape[0],32,32,-1)
x_test=x_test.reshape(x_test.shape[0],32,32,-1)

In [104]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)


x_train shape: (50000, 32, 32, 3)
x_test shape: (10000, 32, 32, 3)
y_train shape: (50000,)
y_test shape: (10000,)


In [105]:
#x_train_5_9=x_train[y_train>=5]

In [106]:
#x_train_5_9

In [107]:
y_train

array([6, 9, 9, ..., 9, 1, 1], dtype=uint8)

In [108]:
y_train_0_4=y_train[y_train<5]

In [109]:
y_train_0_4

array([4, 1, 1, ..., 2, 1, 1], dtype=uint8)

In [110]:
y_test

array([3, 8, 8, ..., 5, 1, 7])

In [111]:
y_train.shape

(50000,)

In [112]:
[y_test <5]

[array([ True, False, False, ..., False,  True, False])]

In [113]:
x_train_0_4=x_train[y_train<5]
x_test_0_4=x_test[y_test<5]
y_train_0_4=y_train[y_train<5]
y_test_0_4=y_test[y_test<5]
x_train_5_9=x_train[y_train>=5]
x_test_5_9=x_test[y_test>=5]
y_train_5_9=y_train[y_train>=5]-5
y_test_5_9=y_test[y_test>=5]-5

In [114]:
x_test_5_9.shape

(5000, 32, 32, 3)

In [115]:
y_test_5_9.shape

(5000,)

In [116]:
#Prep the Data
x_train_0_4 = x_train_0_4.astype('float32')
x_test_0_4 = x_test_0_4.astype('float32')
x_train_5_9 = x_train_5_9.astype('float32')
x_test_5_9 = x_test_5_9.astype('float32')

x_train_0_4= x_train_0_4 / 255 #Done for normalization of data, 
x_test_0_4 = x_test_0_4/255
x_train_5_9 = x_train_5_9 / 255 #Done for normalization of data, 
x_test_5_9 = x_test_5_9 /255

In [117]:
x_test_5_9.shape

(5000, 32, 32, 3)

### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [118]:
from keras.utils import np_utils
y_train_0_4 = np_utils.to_categorical(y_train_0_4, 5)
y_test_0_4 = np_utils.to_categorical(y_test_0_4, 5)
y_train_5_9 = np_utils.to_categorical(y_train_5_9, 5)
y_test_5_9 = np_utils.to_categorical(y_test_5_9, 5)

In [119]:
y_train_0_4,y_test_0_4,y_train_5_9,y_test_5_9

(array([[0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        ...,
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        ...,
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.]], dtype=float32), array([[0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        ...,
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1.]], dtype=float32), array([[0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        ...,
        [0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.]], dtype=float32))

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [129]:
# Importing the required Keras modules containing model and layers
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D,Activation,BatchNormalization
from keras.layers import Flatten, Reshape
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
import tensorflow as tf

# Creating a Sequential Model and adding the layers


In [130]:

    # Define model
model2 = Sequential()

    # 1st Conv Layer
model2.add(Convolution2D(32, 3, 3, input_shape=(32, 32, 3))) 
model2.add(Activation('relu'))

    # 2nd Conv Layer
model2.add(Convolution2D(32, 3, 3))
model2.add(Activation('relu'))
# Max Pooling
model2.add(MaxPooling2D(pool_size=(2,2)))
    
    # Dropout
model2.add(Dropout(0.25))


    # Fully Connected Layer
model2.add(Flatten())
model2.add(Dense(128))
model2.add(Activation('relu'))
model2.add(Dense(1024, activation='relu'))
model2.add(BatchNormalization())
model2.add(Dropout(0.5))
model2.add(Dense(512, activation='relu'))
model2.add(BatchNormalization())
model2.add(Dropout(0.5))
    # Prediction Layer
model2.add(Dense(5))
model2.add(Activation('softmax'))
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

    # Loss and Optimizer
model2.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])


C:\Users\neeha\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(32, 32, 3...)`
  
C:\Users\neeha\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  # Remove the CWD from sys.path while we load stuff.


In [131]:
# Train the model2 
model2.fit(x_train_0_4, y_train_0_4, batch_size=500, nb_epoch=20, 
              validation_data=(x_test_0_4, y_test_0_4))

C:\Users\neeha\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


Train on 25000 samples, validate on 5000 samples
Epoch 1/20
25000/25000 [==============================] - 25s 1ms/step - loss: 2.0523 - acc: 0.3487 - val_loss: 1.3707 - val_acc: 0.5032
Epoch 2/20
25000/25000 [==============================] - 23s 906us/step - loss: 1.6543 - acc: 0.4432 - val_loss: 1.2245 - val_acc: 0.5126
Epoch 3/20
25000/25000 [==============================] - 23s 902us/step - loss: 1.4295 - acc: 0.5038 - val_loss: 1.2755 - val_acc: 0.5336
Epoch 4/20
25000/25000 [==============================] - 23s 910us/step - loss: 1.3055 - acc: 0.5346 - val_loss: 1.0620 - val_acc: 0.5718
Epoch 5/20
25000/25000 [==============================] - 23s 902us/step - loss: 1.1947 - acc: 0.5634 - val_loss: 1.0111 - val_acc: 0.6072
Epoch 6/20
25000/25000 [==============================] - 23s 908us/step - loss: 1.1198 - acc: 0.5896 - val_loss: 0.9566 - val_acc: 0.6156
Epoch 7/20
25000/25000 [==============================] - 23s 914us/step - loss: 1.0655 - acc: 0.6083 - val_loss: 0.950

In [132]:
#Testing the model on train set
score = model2.evaluate(x_train_0_4, y_train_0_4)
print('Train loss for 0-4:', score[0])
print('Train accuracy 0-4:', score[1])

25000/25000 [==============================] - 12s 466us/step
Train loss for 0-4: 0.493346595249176
Train accuracy 0-4: 0.81064


In [133]:
#Testing the model on train set
score = model2.evaluate(x_test_0_4, y_test_0_4)
print('Test loss 0-4:', score[0])
print('Test accuracy 0-4:', score[1])

5000/5000 [==============================] - 2s 412us/step
Test loss 0-4: 0.7199946382522583
Test accuracy 0-4: 0.7324


### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [134]:
#Freezing layers in the model which don't have 'dense' in their name
for layer in model2.layers:
  if('dense' not in layer.name): #prefix detection to freeze layers which does not have dense
    #Freezing a layer
    layer.trainable = False

#Module to print colourful statements
from termcolor import colored

#Check which layers have been frozen 
for layer in model2.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

conv2d_13
False
activation_23
False
conv2d_14
False
activation_24
False
max_pooling2d_8
False
dropout_16
False
flatten_6
False
dense_21
True
activation_25
False
dense_22
True
batch_normalization_11
False
dropout_17
False
dense_23
True
batch_normalization_12
False
dropout_18
False
dense_24
True
activation_26
False


### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [139]:
model2.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

model2.fit(x_train_5_9, y_train_5_9, batch_size=500, nb_epoch=50, 
              validation_data=(x_test_5_9, y_test_5_9))

C:\Users\neeha\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  after removing the cwd from sys.path.


Train on 25000 samples, validate on 5000 samples
Epoch 1/50
25000/25000 [==============================] - 16s 637us/step - loss: 0.1710 - acc: 0.9398 - val_loss: 0.8770 - val_acc: 0.7690
Epoch 2/50
25000/25000 [==============================] - 14s 564us/step - loss: 0.1518 - acc: 0.9442 - val_loss: 1.0168 - val_acc: 0.7494
Epoch 3/50
25000/25000 [==============================] - 14s 563us/step - loss: 0.1490 - acc: 0.9463 - val_loss: 0.8900 - val_acc: 0.7732
Epoch 4/50
25000/25000 [==============================] - 14s 558us/step - loss: 0.1414 - acc: 0.9480 - val_loss: 0.9840 - val_acc: 0.7614
Epoch 5/50
25000/25000 [==============================] - 14s 562us/step - loss: 0.1376 - acc: 0.9502 - val_loss: 0.8965 - val_acc: 0.7768
Epoch 6/50
25000/25000 [==============================] - 14s 560us/step - loss: 0.1255 - acc: 0.9554 - val_loss: 1.0243 - val_acc: 0.7594
Epoch 7/50
25000/25000 [==============================] - 14s 558us/step - loss: 0.1212 - acc: 0.9553 - val_loss: 1.0

In [140]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
activation_23 (Activation)   (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
activation_24 (Activation)   (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 6272)              0         
__________

In [141]:
#Testing the model on train set
score = model2.evaluate(x_train_5_9, y_train_5_9)
print('Train loss for 0-4:', score[0])
print('Train accuracy 0-4:', score[1])

25000/25000 [==============================] - 11s 423us/step
Train loss for 0-4: 0.041335463415710254
Train accuracy 0-4: 0.9842


In [142]:
#Testing the model on train set
score = model2.evaluate(x_test_5_9, y_test_5_9)
print('Test loss 0-4:', score[0])
print('Test accuracy 0-4:', score[1])

5000/5000 [==============================] - 2s 435us/step
Test loss 0-4: 1.383478640732169
Test accuracy 0-4: 0.8062


In [30]:
x_train_5_9.shape

(25000, 32, 32, 3)

In [31]:
y_train_5_9.shape

(25000, 5)

In [32]:
x_test_5_9.shape

(5000, 32, 32, 3)

In [33]:
y_test_5_9.shape

(5000, 5)

# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [72]:
from sklearn.datasets import fetch_20newsgroups

In [73]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [74]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [75]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [76]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2], dtype=int64)

In [77]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [78]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [79]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# use CountVectorizer to create document-term matrices from X_train and X_test
vect = CountVectorizer()

In [82]:
X_train = vect.fit_transform(twenty_train.data)
y_train = twenty_train.target

In [83]:
X_test = vect.transform(twenty_test.data)
y_test = twenty_test.target

In [62]:
# all 100 features
print( vect.get_feature_names())

['data', 'descr', 'filenames', 'target', 'target_names']


In [94]:
X_train.data

array([2, 1, 1, ..., 1, 2, 1], dtype=int64)

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [64]:
from sklearn.linear_model import LogisticRegression

In [65]:
LR= LogisticRegression()

In [89]:
LR.fit(X_train,y_train )

C:\Users\neeha\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\neeha\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [90]:
y_pred_class_LR = LR.predict(X_test)


In [91]:
from sklearn.metrics import accuracy_score

In [92]:
accuracy_score(y_test,y_pred_class_LR)

0.9007989347536618

In [93]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred_class_LR))

              precision    recall  f1-score   support

           0       0.93      0.81      0.86       319
           1       0.88      0.96      0.92       389
           2       0.93      0.84      0.88       396
           3       0.88      0.98      0.93       398

   micro avg       0.90      0.90      0.90      1502
   macro avg       0.90      0.90      0.90      1502
weighted avg       0.90      0.90      0.90      1502

